# Análise Experimental Definitiva de Meta-heurísticas: da Visão Geral ao Comportamento Detalhado

Este notebook apresenta uma análise completa e multi-nível do desempenho de quatro algoritmos de otimização. A análise progride desde uma visão estatística agregada até um mergulho profundo na trajetória de busca de uma única execução, oferecendo insights sobre a eficácia e o comportamento de cada método.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Configuração visual dos gráficos
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams.update({
    'figure.figsize': (15, 9),
    'figure.dpi': 110,
    'axes.titlesize': 18,
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})
print("Bibliotecas importadas e configurações de plotagem aplicadas.")

### 1. Carregamento e Preparação dos Dados

In [ ]:
def load_and_clean_data(base_path, max_time=2.0):
    """Carrega e limpa os dados de resultado final."""
    search_pattern = os.path.join(base_path, "run_*", "*", "*", "saida_*.txt")
    file_paths = glob.glob(search_pattern)
    if not file_paths: return pd.DataFrame()
    all_data = []
    abs_base_path = os.path.abspath(base_path)
    for file_path in file_paths:
        try:
            relative_path = os.path.relpath(file_path, abs_base_path)
            parts = relative_path.split(os.sep)
            if len(parts) != 4: continue
            df_temp = pd.read_csv(file_path, sep=' ', header=None, names=['valor', 'tempo'])
            df_temp['run'] = int(parts[0].replace("run_", ""))
            df_temp['algoritmo'] = parts[1]
            df_temp['tipo_instancia'] = parts[2]
            df_temp['tamanho_instancia'] = int(parts[3].replace("saida_", "").replace(".txt", ""))
            all_data.append(df_temp)
        except: pass
    df_raw = pd.concat(all_data, ignore_index=True)
    df_cleaned = df_raw[(df_raw['valor'] >= 0) & (df_raw['tempo'] <= max_time)].copy()
    return df_cleaned

def load_trajectory_data(base_path):
    """Carrega os dados de trajetória/convergência por iteração."""
    search_pattern = os.path.join(base_path, "run_*", "*", "*", "conv_*.txt")
    file_paths = glob.glob(search_pattern)
    if not file_paths: return pd.DataFrame()

    all_data = []
    abs_base_path = os.path.abspath(base_path)
    for file_path in file_paths:
        try:
            relative_path = os.path.relpath(file_path, abs_base_path)
            parts = relative_path.split(os.sep)
            if len(parts) != 4: continue
            df_temp = pd.read_csv(file_path, sep=' ', header=None, names=['iteracao', 'valor'])
            df_temp['run'] = int(parts[0].replace("run_", ""))
            df_temp['algoritmo'] = parts[1]
            df_temp['tipo_instancia'] = parts[2]
            fname_parts = parts[3].replace(".txt", "").split('_')
            df_temp['cenario'] = int(fname_parts[1].replace('s', ''))
            df_temp['tamanho_instancia'] = int(fname_parts[3].replace('z', ''))
            all_data.append(df_temp)
        except: pass
    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Carregando os dados
results_path = '/home/mjf30/otmgraf/Otimizacao-Grafos/resultados'
convergence_path = '/home/mjf30/otmgraf/Otimizacao-Grafos/convergencia'

df_results = load_and_clean_data(results_path)
df_trajectory = load_trajectory_data(convergence_path)

if not df_results.empty: print(f"Dados de resultados finais carregados: {len(df_results)} registros.")
if not df_trajectory.empty: print(f"Dados de trajetória carregados: {len(df_trajectory)} registros.")

### 2. Análise Agregada e Visualizações

In [ ]:
if not df_results.empty:
    summary_stats = df_results.groupby(['algoritmo', 'tipo_instancia', 'tamanho_instancia']).agg(
        valor_medio=('valor', 'mean'),
        valor_std=('valor', 'std'),
        tempo_medio=('tempo', 'mean'),
        tempo_std=('tempo', 'std')
    ).reset_index()
    summary_stats['cv_valor_%'] = (summary_stats['valor_std'] / (summary_stats['valor_medio'].abs() + 1e-9)) * 100
    summary_stats['cv_tempo_%'] = (summary_stats['tempo_std'] / (summary_stats['tempo_medio'].abs() + 1e-9)) * 100
    
    # Boxplots para consistência
    instance_types_in_data = sorted(df_results['tipo_instancia'].unique())
    for inst_type in instance_types_in_data:
        print(f"\n{'='*25} ANÁLISE VISUAL PARA: {inst_type.upper()} {'='*25}")
        plt.figure()
        sns.boxplot(data=df_results[df_results['tipo_instancia'] == inst_type], x='tamanho_instancia', y='valor', hue='algoritmo')
        plt.title(f'Consistência dos Resultados - {inst_type}', weight='bold')
        plt.ylabel('Valor da Solução (Fitness)'); plt.xlabel('Tamanho da Instância')
        plt.legend(title='Algoritmo', bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.tight_layout(); plt.show()

### 3. Análise de Trajetória de Busca (Execução Única)

In [ ]:
if not df_trajectory.empty:
    target_type = 'fully_correlated_sc'
    target_size = 1000
    target_run = 1

    data_traj = df_trajectory[
        (df_trajectory['tipo_instancia'] == target_type) &
        (df_trajectory['tamanho_instancia'] == target_size) &
        (df_trajectory['run'] == target_run)
    ]

    if not data_traj.empty:
        plt.figure()
        sns.lineplot(data=data_traj, x='iteracao', y='valor', hue='algoritmo', 
                     style='algoritmo', markers=False, drawstyle='steps-post', linewidth=2.5)
        
        plt.title(f'Trajetória de Busca por Iteração (Execução Única)\n{target_type} - {target_size} - Run {target_run}', weight='bold')
        plt.xlabel('Número de Iterações')
        plt.ylabel('Valor da Melhor Solução Encontrada')
        plt.legend(title='Algoritmo')
        plt.grid(True, which='both', linestyle='--')
        plt.show()
    else:
        print(f"Nenhum dado de trajetória encontrado para {target_type}, {target_size}, Run {target_run}.")

### 4. Análise de Trade-Off e Significância Estatística

In [ ]:
def identify_pareto(scores):
    population_size = scores.shape[0]
    pareto_front = np.ones(population_size, dtype=bool)
    for i in range(population_size):
        for j in range(population_size):
            if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                pareto_front[i] = False
                break
    return pareto_front

if not df_results.empty:
    instances_to_analyze = [
        ('correlated_sc', 500),
        ('fully_correlated_sc', 1000),
        ('not_correlated_sc', 800)
    ]

    # Gráficos de Pareto
    for inst_type, inst_size in instances_to_analyze:
        data_pareto = df_results[(df_results['tipo_instancia'] == inst_type) & (df_results['tamanho_instancia'] == inst_size)]
        if data_pareto.empty: continue
        plt.figure()
        sns.scatterplot(data=data_pareto, x='tempo', y='valor', hue='algoritmo', alpha=0.5, s=60, style='algoritmo')
        scores = np.array([data_pareto['valor'], -data_pareto['tempo']]).T
        global_pareto_mask = identify_pareto(scores)
        global_pareto_points = data_pareto[global_pareto_mask]
        plt.scatter(global_pareto_points['tempo'], global_pareto_points['valor'], s=250, facecolors='none', edgecolors='red', marker='o', linewidth=2.5, label='Fronteira Pareto Global', zorder=10)
        plt.title(f'Fronteira de Pareto: Qualidade vs. Tempo\n{inst_type} - {inst_size}', weight='bold')
        plt.xlabel('Tempo de Execução (s)'); plt.ylabel('Valor da Solução (Fitness)')
        plt.legend(title='Algoritmo / Destaque', bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.grid(True, which='both', linestyle='--')
        plt.tight_layout(); plt.show()

    # Testes Estatísticos
    for inst_type, inst_size in instances_to_analyze:
        print(f"\n{'='*20} Análise Estatística para: {inst_type.upper()}, Tamanho: {inst_size} {'='*20}")
        data_for_test = df_results[(df_results['tipo_instancia'] == inst_type) & (df_results['tamanho_instancia'] == inst_size)]
        if len(data_for_test['algoritmo'].unique()) < 2: continue
        samples = [group['valor'].values for name, group in data_for_test.groupby('algoritmo')]
        f_statistic, p_value_anova = stats.f_oneway(*samples)
        print(f"--- Teste ANOVA --- -> P-valor: {p_value_anova:.4g}")
        if p_value_anova < 0.05:
            print("-> Conclusão ANOVA: Existe uma diferença estatisticamente significante.")
            tukey_results = pairwise_tukeyhsd(endog=data_for_test['valor'], groups=data_for_test['algoritmo'], alpha=0.05)
            print("--- Teste Post-Hoc de Tukey HSD ---")
            print(tukey_results)
        else:
            print("-> Conclusão ANOVA: NÃO há evidência de diferença estatisticamente significante.")

### 5. Tabelas de Resumo para Relatório

In [ ]:
if 'summary_stats' in locals() and not summary_stats.empty:
    # --- Tabela Resumo (Formato Texto Simples) ---
    target_size_report = 1000
    data_report = summary_stats[summary_stats['tamanho_instancia'] == target_size_report]
    
    print(f"\n{'='*60}")
    print(f"### RELATÓRIO CONSOLIDADO PARA INSTÂNCIAS DE TAMANHO N = {target_size_report} ###")
    print(f"{'='*60}\n")

    for inst_type in sorted(data_report['tipo_instancia'].unique()):
        print(f"--- TIPO DE INSTÂNCIA: {inst_type.upper()} ---\n")
        subset = data_report[data_report['tipo_instancia'] == inst_type].sort_values(by='valor_medio', ascending=False)
        
        for _, row in subset.iterrows():
            print(f"** {row['algoritmo'].upper()} **")
            print(f"    - Média das soluções: {row['valor_medio']:.0f}")
            print(f"    - Coeficiente de Variação (Soluções): {row['cv_valor_%']:.2f}%")
            print(f"    - Média dos Tempos: {row['tempo_medio']:.3f} seg")
            print(f"    - Coeficiente de Variação (Tempo): {row['cv_tempo_%']:.1f}%\n")
        print("---\n")